

```
README:
Code was adapted from CSE 354 Assignment 2 DAN as well as from Sije's code.
Also uses the GloVe vectors which are part of the Torch library.
Final models are located here: https://drive.google.com/drive/folders/13T_HyJBF7Jj-_K2bJ65hB8U5i7He1gOF?usp=share_link
There is a code segment commented out that allows for a pre-trained model to be loaded instead.

```



In [62]:
import torch
import pandas as pd
from torch.utils.data import Dataset, TensorDataset, DataLoader, ConcatDataset
from torchtext.vocab.vectors import GloVe
import torch.nn as nn
from tqdm import tqdm
import requests
import zipfile
import re
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import itertools


In [63]:
#from google.colab import drive
#drive.mount('/content/drive',force_remount=True)
#%cd "/content/drive/MyDrive/CSE_354/Final_Project"

In [64]:
request = requests.get("https://github.com/MHDBST/PerSenT/archive/refs/heads/main.zip")
with open("data.zip", "wb") as file:
    file.write(request.content)

# Unzip data
with zipfile.ZipFile('data.zip') as zip:
    zip.extractall('data')

In [65]:
# Use pre-trained GloVe embeddings
embed_dims = 100
glove = GloVe(name='6B', dim=embed_dims)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [66]:
class DAN(nn.Module):
  def __init__(self, input_dim: int, num_layers: int, dropout: float = 0.2, device = 'cpu'):
    super(DAN, self).__init__()
    self.input_dim = input_dim
    self.num_layers = num_layers
    self.dropout = dropout
    self.device = device

    layer_list = [nn.Linear(self.input_dim, self.input_dim, device=self.device) for i in range(self.num_layers)]
    self.module_list = nn.ModuleList(layer_list)

  def forward(self, vector_sequence: torch.Tensor, sequence_mask: torch.Tensor, training=False):
    batch_size = vector_sequence.size(dim=0)
    sequence_size = vector_sequence.size(dim=1)

    #print("vector_sequence.shape:", vector_sequence.shape)

    # Apply dropout if training is set to True
    if training:
      dropout_mask = torch.bernoulli(torch.full([batch_size, sequence_size, 1], 1 - self.dropout, device=self.device))
      #print("dropout_mask.shape:", dropout_mask.shape)
      vector_sequence = torch.nan_to_num(torch.mul(dropout_mask, vector_sequence))

    # Average word embedding vectors
    #average_vector = torch.mean(vector_sequence, dim=1)
    average_vector = torch.sum(vector_sequence, dim=1) / sequence_mask.reshape(-1, 1)
    #print("average_vector.shape:", average_vector.shape)

    # Apply feed-forward layers
    for i in range(self.num_layers - 1):
      l = self.module_list[i]
      average_vector = nn.functional.relu(average_vector)
    output_vector = self.module_list[self.num_layers - 1](average_vector) # Apply final layer without ReLU

    # Return output vector
    return output_vector

In [67]:
class DANClassifier(nn.Module):
  def __init__(self, embedding_dim: int, num_layers: int, num_classes: int, device: str = 'cpu'):
    super(DANClassifier, self).__init__()
    
    self.dan_layer = DAN(embedding_dim, num_layers, device=device).to(device)
    self.classification_layer = nn.Linear(embedding_dim, num_classes).to(device)

  def forward(self, inputs, seq_mask, training=False):
    dan_output = self.dan_layer(inputs, seq_mask, training=training)
    #print("dan_output.shape:", dan_output.shape)
    classification_output = self.classification_layer(dan_output)
    #print("classification_output.shape:", classification_output.shape)
    softmax = nn.Softmax(dim=1)
    return softmax(classification_output)


In [68]:
def get_paragraph_embeddings(paragraph_list, max_length = 512):
    # Get embeddings for a list of paragraphs
    token_embeds_list = list()
    sequence_mask_list = list()
    for i in range(len(paragraph_list)):
        # Set all token sequences to the same length, using padding if necessary
        tokens = [t.lower() for t in paragraph_list[i].split()]
        sequence_mask = len(tokens)
        if len(tokens) > max_length:
          tokens = tokens[0:max_length]
        elif len(tokens) < max_length:
          tokens.extend([""] * (max_length - len(tokens)))
        input_embeds = glove.get_vecs_by_tokens(tokens)

        # Get sentiment
        #label = torch.tensor(label_map[self.sentiments[i]], dtype=torch.long)

        token_embeds_list.append(input_embeds)
        sequence_mask_list.append(sequence_mask)
    
    return token_embeds_list, sequence_mask_list

# This method is responsible for reading the CSV file containing paragraph data,
# coverting them to word embeddings, and returning the embeddings along with the sentiment label.
def get_dataset(path, max_length = 512, batch_size: int = 256):
    # seperate data by columns.
    data = pd.read_csv(path, delimiter=',')
    data.dropna(subset=['DOCUMENT','TRUE_SENTIMENT'], inplace=False)

    # Collect paragraphs and their sentiments
    paragraphs = []
    sentiments = []
    
    for _, row in data.iterrows():
        #devide each paragraph from document.
        doc_paragraphs = row['DOCUMENT'].split('\n')
        for i in range(len(doc_paragraphs)):
            if(i >=16):
              break
            paragraph_col = f'Paragraph{i}'
            if pd.notna(row[paragraph_col]):
                #store its paragraph
                paragraphs.append(doc_paragraphs[i])
                #get each sentiments 
                sentiments.append(row[paragraph_col])
            # if there is one paragraph, then that paragraph is the document sentiment.
            elif (len(doc_paragraphs) == 1): 
                sentiments.append(row['TRUE_SENTIMENT'])  
                paragraphs.append(doc_paragraphs[i])

    # Generate embeddings for all inputs
    """
    label_map = {'Positive': 0, 'Neutral': 1, 'Negative': 2}
    token_embeds_list = list()
    sequence_mask_list = list()
    sentiment_list = list()
    for i in range(len(paragraphs)):
        # Set all token sequences to the same length, using padding if necessary
        tokens = [t.lower() for t in paragraphs[i].split()]
        sequence_mask = len(tokens)
        if len(tokens) > max_length:
          tokens = tokens[0:max_length]
        elif len(tokens) < max_length:
          tokens.extend([""] * (max_length - len(tokens)))
        input_embeds = glove.get_vecs_by_tokens(tokens)

        # Get sentiment
        #label = torch.tensor(label_map[self.sentiments[i]], dtype=torch.long)

        token_embeds_list.append(input_embeds)
        sequence_mask_list.append(sequence_mask)
        sentiment_list.append(label_map[sentiments[i]])
    #print("Sizes:", torch.stack(token_embeds_list).shape, torch.stack(sentiment_list).shape)
    #print("len(sentiment_list):", len(sentiment_list))
    #print("sentiment_list[0].shape", sentiment_list[0].shape)
    """
    token_embeds_list, sequence_mask_list = get_paragraph_embeddings(paragraphs, max_length=max_length)
    label_map = {'Positive': 0, 'Neutral': 1, 'Negative': 2}
    sentiment_list = [label_map[s] for s in sentiments]

    # Generate weights for each class to balance out frequency
    num_positive = sentiment_list.count(0)
    num_neutral = sentiment_list.count(1)
    num_negative = sentiment_list.count(2)

    pct_positive = num_positive / len(sentiment_list)
    pct_neutral = num_neutral / len(sentiment_list)
    pct_negative = num_negative / len(sentiment_list)

    class_weights = [len(sentiment_list) / num_positive, len(sentiment_list) / num_neutral, len(sentiment_list) / num_negative]
    print(f"Class frequencies: {num_positive} positive, {num_neutral} neutral, {num_negative} negative")
    print(class_weights)

    #print("torch.stack(token_embeds_list).shape:", torch.stack(token_embeds_list).shape)
    #print("torch.tensor(sentiment_list).shape:", torch.tensor(sentiment_list).shape)
    dataset = TensorDataset(torch.stack(token_embeds_list), torch.tensor(sequence_mask_list), torch.tensor(sentiment_list))
    return dataset, torch.tensor(class_weights)


In [76]:
def plot_confusion_matrix(cm, target_names):
    plt.imshow(cm, interpolation='nearest', cmap=plt.get_cmap('Blues'))
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > cm.max() / 2 else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

# Training and validation function for combined datasets
def train_model(model, train_loader, val_loader, class_weights, num_epochs=4, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=4e-5)
    cross_entropy_loss = nn.CrossEntropyLoss(weight=class_weights.float().to(device))
    for epoch in range(num_epochs):
        # Training loop
        model.train()
        epoch_total_loss = 0
        for i, batch in enumerate(tqdm(train_loader)):
            input_embeds, seq_mask, labels = batch
            input_embeds = input_embeds.to(device)
            seq_mask = seq_mask.to(device)
            labels = labels.to(device)

            logits = model(input_embeds, seq_mask, training=True)
            loss = cross_entropy_loss(logits.to(device), labels)
            #print("loss: ", loss)
            epoch_total_loss += loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        epoch_average_loss = epoch_total_loss / len(train_loader)
        print("Average loss:", epoch_average_loss)

        # Validation loop
        true_labels = []
        pred_labels = []
        model.eval()
        total_correct = 0
        for batch in val_loader:
            input_embeds, seq_mask, labels = batch
            input_embeds = input_embeds.to(device)
            seq_mask = seq_mask.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                logits = model(input_embeds, seq_mask, training=False)
                _, predictions = torch.max(logits, dim=1) # argMax
                #print("predictions.shape:", predictions.shape)
                total_correct += torch.sum(predictions == labels).item()
            true_labels.extend(labels.cpu().numpy())
            pred_labels.extend(predictions.cpu().numpy())

        accuracy = total_correct / len(val_loader.dataset)
        report = classification_report(true_labels, pred_labels, target_names=['Positive', 'Neutral', 'Negative'])
        cm = confusion_matrix(true_labels, pred_labels)
        
        print(f'Epoch {epoch + 1}/{num_epochs}, Report: {report}')
        #print('Confusion Matrix:')
        #plot_confusion_matrix(cm, target_names=['Negative', 'Neutral', 'Positive'])

    torch.save(model, "model.pkg")

    return model

In [11]:
train_dataset, training_class_weights = get_dataset('data/PerSenT-main/train.csv', max_length=512, batch_size=256)
val_dataset, val_class_weights = get_dataset('data/PerSenT-main/dev.csv', max_length=512, batch_size=256)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
#train_loader = DataLoader(train_dataset, batch_size=350, shuffle=False)
#val_loader = DataLoader(val_dataset, batch_size=350, shuffle=False)

Class frequencies: 10662 positive, 10971 neutral, 2413 negative
[2.255299193397111, 2.191778324674141, 9.96518856195607]
Class frequencies: 1792 positive, 2007 neutral, 391 negative
[2.338169642857143, 2.0876930742401596, 10.71611253196931]


In [13]:
model = DANClassifier(embedding_dim=embed_dims, num_layers=4, num_classes=3, device=device)
model = train_model(model, train_loader, val_loader, training_class_weights, num_epochs=500)

100%|██████████| 94/94 [00:05<00:00, 18.12it/s]


Average loss: tensor(1.0995, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 1/500, Report:               precision    recall  f1-score   support

    Positive       0.38      0.01      0.01      1792
     Neutral       0.48      0.99      0.65      2007
    Negative       0.00      0.00      0.00       391

    accuracy                           0.48      4190
   macro avg       0.29      0.33      0.22      4190
weighted avg       0.39      0.48      0.31      4190



100%|██████████| 94/94 [00:02<00:00, 31.41it/s]


Average loss: tensor(1.0992, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 2/500, Report:               precision    recall  f1-score   support

    Positive       0.44      0.01      0.02      1792
     Neutral       0.48      0.99      0.65      2007
    Negative       0.00      0.00      0.00       391

    accuracy                           0.48      4190
   macro avg       0.31      0.33      0.22      4190
weighted avg       0.42      0.48      0.32      4190



100%|██████████| 94/94 [00:02<00:00, 40.04it/s]


...

Average loss: tensor(1.0487, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 498/500, Report:               precision    recall  f1-score   support

    Positive       0.54      0.57      0.55      1792
     Neutral       0.55      0.28      0.37      2007
    Negative       0.16      0.54      0.25       391

    accuracy                           0.43      4190
   macro avg       0.42      0.46      0.39      4190
weighted avg       0.51      0.43      0.44      4190



100%|██████████| 94/94 [00:02<00:00, 36.14it/s]


Average loss: tensor(1.0503, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 499/500, Report:               precision    recall  f1-score   support

    Positive       0.54      0.57      0.55      1792
     Neutral       0.55      0.28      0.37      2007
    Negative       0.16      0.54      0.25       391

    accuracy                           0.43      4190
   macro avg       0.42      0.46      0.39      4190
weighted avg       0.51      0.43      0.44      4190



100%|██████████| 94/94 [00:02<00:00, 40.25it/s]


Average loss: tensor(1.0495, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 500/500, Report:               precision    recall  f1-score   support

    Positive       0.54      0.56      0.55      1792
     Neutral       0.55      0.28      0.37      2007
    Negative       0.16      0.54      0.25       391

    accuracy                           0.43      4190
   macro avg       0.42      0.46      0.39      4190
weighted avg       0.51      0.43      0.44      4190



In [70]:
# Uncommon this to load a pre-trained model
#model = torch.load("./model.pkg", map_location=device)

In [73]:
# This method is responsible for predicting the sentiments of each paragraph and document
def predict_sentiments(path):
    # seperate data by columns.
    data = pd.read_csv(path, delimiter=',')
    data.dropna(subset=['DOCUMENT','TRUE_SENTIMENT'], inplace=False)

    # Collect paragraphs and their sentiments
    paragraphs = []
    paragraph_sentiments = []
    documents = []
    document_sentiments = []

    paragraph_data = pd.DataFrame({"PARAGRAPH" : [], "DOCUMENT_ID" : [], "TRUE_SENTIMENT" : [], "PREDICTED_SENTIMENT" : []})
    document_data = pd.DataFrame({"DOCUMENT" : [], "TRUE_SENTIMENT" : [], "PREDICTED_SENTIMENT" : []})
    
    pg_count = 0
    softmax = nn.Softmax(dim=1)
    sentiment_values = ["Positive", "Negative", "Neutral"]
    for index, row in data.iterrows():
        document_paragraph_sentiments = []
        
        #devide each paragraph from document.
        doc_paragraphs = row['DOCUMENT'].split('\n')
        for i in range(len(doc_paragraphs)):
            if(i >=16):
              break
            paragraph_col = f'Paragraph{i}'
            if pd.notna(row[paragraph_col]):
                #store its paragraph
                paragraphs.append(doc_paragraphs[i])
                #get each sentiments 
                paragraph_sentiments.append(row[paragraph_col])
                document_paragraph_sentiments.append(row[paragraph_col])
            # if there is one paragraph, then that paragraph is the document sentiment.
            elif (len(doc_paragraphs) == 1): 
                paragraph_sentiments.append(row['TRUE_SENTIMENT'])  
                paragraphs.append(doc_paragraphs[i])
                document_paragraph_sentiments.append(row['TRUE_SENTIMENT'])

        doc_paragraphs_embeddings, seq_masks = get_paragraph_embeddings(doc_paragraphs, max_length=512)
        with torch.no_grad():
            # Get document and paragraph level sentiment predictions
            logits = model(torch.stack(doc_paragraphs_embeddings).to(device), torch.tensor(seq_masks).to(device))
            predictions = torch.argmax(softmax(logits), dim=1)
            #print(predictions.shape)
            predicted_sentiments = [sentiment_values[i] for i in predictions.tolist()]
            #print(predicted_sentiments)
            predicted_document_sentiment = max(predicted_sentiments, key=predicted_sentiments.count)
            paragraph_sentiments.extend(predicted_sentiments)
            document_sentiments.append(predicted_document_sentiment)

            # Add predictions to dataframe
            if len(doc_paragraphs) == len(document_paragraph_sentiments):
                for i in range(len(doc_paragraphs)):
                    paragraph_data.loc[pg_count] = [doc_paragraphs[i], index, document_paragraph_sentiments[i], predicted_sentiments[i]]
                    pg_count += 1
                document_data.loc[index] = [row['DOCUMENT'], row['TRUE_SENTIMENT'], predicted_document_sentiment]

    # Get dataset statistics
    return document_data, paragraph_data




In [74]:
# Get sentiment predictions on document and paragraph level
document_data, paragraph_data = predict_sentiments("data/PerSenT-main/fixed_test.csv")

document_data.to_csv("document_preds.csv")
paragraph_data.to_csv("paragraphs_preds.csv")

# Print performance statistics
document_report = classification_report(document_data["TRUE_SENTIMENT"], document_data["PREDICTED_SENTIMENT"], target_names=['Positive', 'Neutral', 'Negative'])
paragraph_report = classification_report(paragraph_data["TRUE_SENTIMENT"], paragraph_data["PREDICTED_SENTIMENT"], target_names=['Positive', 'Neutral', 'Negative'])
print(document_report)
print(paragraph_report)

              precision    recall  f1-score   support

    Positive       0.14      0.31      0.19       120
     Neutral       0.41      0.44      0.42       285
    Negative       0.53      0.27      0.35       339

    accuracy                           0.34       744
   macro avg       0.36      0.34      0.32       744
weighted avg       0.42      0.34      0.35       744

              precision    recall  f1-score   support

    Positive       0.14      0.30      0.19       971
     Neutral       0.48      0.39      0.43      2873
    Negative       0.41      0.29      0.34      2156

    accuracy                           0.34      6000
   macro avg       0.34      0.33      0.32      6000
weighted avg       0.40      0.34      0.36      6000



In [75]:
document_data[(document_data["TRUE_SENTIMENT"] != document_data["PREDICTED_SENTIMENT"])]

,DOCUMENT,TRUE_SENTIMENT,PREDICTED_SENTIMENT
0,term extension of government funding that woul...,Neutral,Negative
1,At a press conference in June a reporter ask...,Neutral,Negative
2,Trump Xi present united front despite differ...,Positive,Negative
3,The Latest on President Donald Trump in Florid...,Neutral,Negative
6,The U . S . Senate on Wednesday took a step to...,Neutral,Negative
...,...,...,...
822,"WASHINGTON (AP) â Donald Trump 's ""opaque"" ...",Negative,Neutral
823,President Donald Trump's rally in Missouri thi...,Positive,Negative
824,WASHINGTON (Reuters) - Major automakers suppl...,Positive,Negative
825,President Donald Trump says the Justice Depar...,Negative,Neutral
